# Deep generative modeling for SARS-CoV-2 drug discovery: non-neural baselines

Training a Hidden Markov Model as a non-neural baseline.

Jacqueline R. M. A. Maasch | March 2022

## Preamble

In [1]:
%%capture
# Install Therapeutics Data Commons.
#!python3.8 -m pip install PyTDC

# Install pomegranate for HMM model.
#!python3.8 -m pip install pomegranate

# Needed to resolve error in loading pomegranate.
#!python3.8 -m pip install numpy --upgrade numpy

# Necessary prior to installing moses.
#!python3.8 -m conda install -yq -c rdkit rdkit
#!python3.8 -m pip install rdkit-pypi # << This one worked on my machine.

# Install moses.
#!python3.8 -m pip install molsets



# Enviroment setup - Ian on g2 cluster
# conda create -n cs6785_py38 python=3.8
# conda install -c rdkit rdkit
# pip install pomegranate # version: 0.14.8
# conda install lfs-git
# clone the repository from the Github
# remove the pomegranate in the sertup.py
# run python setup.py install
# conda install pandas
# pip install PyTDC 
# conda install requests
# pip install rdkit-pypi

In [2]:
# Importations.
import pandas as pd
import numpy as np
import tdc
from tdc.single_pred import HTS
from moses_hmm import HMM
from moses_ngram import NGram
from moses_combinatorial import CombinatorialGenerator
import sys
from importlib.metadata import version
from datetime import datetime

# View versioning.
print("\npython version:", sys.version)
print("\n--- LIBRARY VERSIONS ---")
print("tdc:         ", version("PyTDC"))
print("numpy:       ", version("numpy"))
print("pandas:      ", version("pandas"))
print("moses:       ", version("molsets"))
print("rdkit:       ", version("rdkit-pypi"))
print("pomegranate: ", version("pomegranate"))


python version: 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:18) 
[GCC 10.3.0]

--- LIBRARY VERSIONS ---
tdc:          0.3.6
numpy:        1.22.3
pandas:       1.1.3
moses:        1.0
rdkit:        2022.3.1
pomegranate:  0.14.8


## Define functions

## Load data

Data from the [Therapeutics Data Commons](https://tdcommons.ai/single_pred_tasks/hts/).

**SARS-CoV-2 In Vitro, Touret et al.**
- Dataset Description: An in-vitro screen of the Prestwick chemical library composed of 1,480 approved drugs in an infected cell-based assay. From MIT AiCures.
- Task Description: Binary classification. Given a drug SMILES string, predict its activity against SARSCoV2.
- Dataset Statistics: 1,480 drugs.

**SARS-CoV-2 3CL Protease, Diamond.**
- Dataset Description: A large XChem crystallographic fragment screen against SARS-CoV-2 main protease at high resolution. From MIT AiCures.
- Task Description: Binary classification. Given a drug SMILES string, predict its activity against SARSCoV2 3CL Protease.
- Dataset Statistics: 879 drugs.

In [3]:
# Default random state seed for TDC.
# Read about split methods here: https://tdcommons.ai/functions/data_split/
# Consider scaffold split over random split in the future.
random_seed = 42

# SARS-CoV-2 In Vitro, Touret et al.
data_touret = HTS(name = "SARSCoV2_Vitro_Touret")
split_touret = data_touret.get_split(method = "random", 
                                     seed = random_seed, 
                                     frac = [0.7, 0.1, 0.2])

# SARS-CoV-2 3CL Protease, Diamond.
data_diamond = HTS(name = "SARSCoV2_3CLPro_Diamond")
split_diamond = data_diamond.get_split(method = "random", 
                                       seed = random_seed, 
                                       frac = [0.7, 0.1, 0.2])

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


In [4]:
# Explore data.
print("dataset datatype:", type(data_touret))
print("data split datatype:", type(split_touret), "\n")
display(data_touret)
print()
display(split_touret)

dataset datatype: <class 'tdc.single_pred.hts.HTS'>
data split datatype: <class 'dict'> 



{'train':       Drug_ID                                               Drug  Y
 0           0                       CCOc1ccc2nc(S(N)(=O)=O)sc2c1  1
 1           1  C[C@]12C/C(=C/O)C(=O)C[C@@H]1CC[C@@H]1[C@@H]2C...  1
 2           2               Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O.Cl  1
 3           3  CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](O)CC[C...  1
 4           4  C=C1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3C...  1
 ...       ...                                                ... ..
 1034     1477                        O=C(CCCCCCC(=O)Nc1ccccc1)NO  0
 1035     1478        COc1ccccc1OCCNCC(O)COc1cccc2[nH]c3ccccc3c12  0
 1036     1481  Clc1ccc(C(Cn2ccnc2)OCc2csc3c(Cl)cccc23)c(Cl)c1...  0
 1037     1482  CCSc1ccc2c(c1)N(CCCN1CCN(C)CC1)c1ccccc1S2.O=C(...  0
 1038     1483  C=Cc1c(C)c2cc3nc(cc4[nH]c(cc5nc(cc1[nH]2)C(C)=...  0
 
 [1039 rows x 3 columns],
 'valid':      Drug_ID                                               Drug  Y
 0        581              CC(=O)OCC(CCn1cnc2cnc(N)nc21)CO

## Process data

### Extract splits

In [5]:
# Extract training / validation / testing sets.
train_touret = split_touret.get("train")
val_touret = split_touret.get("valid")
test_touret = split_touret.get("test")

train_diamond = split_diamond.get("train")
val_diamond = split_diamond.get("valid")
test_diamond = split_diamond.get("test")

In [6]:
# Explore data splits.
print("\n~~~~~ TOURET: SARS-CoV-2 In Vitro, Touret et al. ~~~~~\n")
print("\n--- TRAINING SPLIT ---\n")
print(train_touret.Y.value_counts())
print(train_touret.Y.value_counts(normalize = True), "\n")
print(train_touret.info())
display(train_touret.head())
print("\n--- VALIDATION SPLIT ---\n")
print(val_touret.Y.value_counts())
print(val_touret.Y.value_counts(normalize = True), "\n")
print(val_touret.info())
display(val_touret.head())
print("\n--- TEST SPLIT ---\n")
print(test_touret.Y.value_counts())
print(test_touret.Y.value_counts(normalize = True), "\n")
print(test_touret.info())
display(test_touret.head())

# Explore data splits.
print("\n~~~~~ DIAMOND: SARS-CoV-2 3CL Protease, Diamond et al. ~~~~~\n")
print("\n--- TRAINING SPLIT ---\n")
print(train_diamond.Y.value_counts())
print(train_diamond.Y.value_counts(normalize = True), "\n")
print(train_diamond.info())
display(train_diamond.head())
print("\n--- VALIDATION SPLIT ---\n")
print(val_diamond.Y.value_counts())
print(val_diamond.Y.value_counts(normalize = True), "\n")
print(val_diamond.info())
display(val_diamond.head())
print("\n--- TEST SPLIT ---\n")
print(test_diamond.Y.value_counts())
print(test_diamond.Y.value_counts(normalize = True), "\n")
print(test_diamond.info())
display(test_diamond.head())


~~~~~ TOURET: SARS-CoV-2 In Vitro, Touret et al. ~~~~~


--- TRAINING SPLIT ---

0    977
1     62
Name: Y, dtype: int64
0    0.940327
1    0.059673
Name: Y, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  1039 non-null   int64 
 1   Drug     1039 non-null   object
 2   Y        1039 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 24.5+ KB
None


,Drug_ID,Drug,Y
0,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,1
1,1,C[C@]12C/C(=C/O)C(=O)C[C@@H]1CC[C@@H]1[C@@H]2C...,1
2,2,Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O.Cl,1
3,3,CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](O)CC[C...,1
4,4,C=C1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3C...,1



--- VALIDATION SPLIT ---

0    141
1      7
Name: Y, dtype: int64
0    0.952703
1    0.047297
Name: Y, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  148 non-null    int64 
 1   Drug     148 non-null    object
 2   Y        148 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ KB
None


,Drug_ID,Drug,Y
0,581,CC(=O)OCC(CCn1cnc2cnc(N)nc21)COC(C)=O,0
1,375,CC(=O)S[C@@H]1CC2=CC(=O)CC[C@]2(C)[C@H]2CC[C@@...,0
2,703,Nc1nc2c(ncn2CCC(CO)CO)c(=O)[nH]1,0
3,1039,NC(=O)N1c2ccccc2C=Cc2ccccc21,0
4,610,O=C(O)CCc1nc(-c2ccccc2)c(-c2ccccc2)o1,0



--- TEST SPLIT ---

0    278
1     19
Name: Y, dtype: int64
0    0.936027
1    0.063973
Name: Y, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  297 non-null    int64 
 1   Drug     297 non-null    object
 2   Y        297 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 7.1+ KB
None


...

,Drug_ID,Drug,Y
0,1,CC(=O)NCCc1c[nH]c2ccc(F)cc12,1
1,2,NC(=O)[C@H]1CCC[C@H]1c1ccsc1,1
2,3,CN1CCCc2ccc(S(N)(=O)=O)cc21,1
3,4,CC(=O)Nc1ccc(Oc2ncccn2)cc1,1
4,6,O=C(CCl)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1,1



--- VALIDATION SPLIT ---

0    78
1    10
Name: Y, dtype: int64
0    0.886364
1    0.113636
Name: Y, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  88 non-null     int64 
 1   Drug     88 non-null     object
 2   Y        88 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 2.2+ KB
None


,Drug_ID,Drug,Y
0,498,CC(=O)NCC1(c2ccccc2)CCOCC1,0
1,524,Nc1cc(C(F)(F)F)ccc1N1CCCCC1,0
2,410,Cn1cc(Oc2ncncc2Cl)cn1,0
3,233,CC1(C(N)=O)CCCN1,0
4,229,O=C(CCl)N1CCN(Cc2c(F)cccc2Cl)CC1,0



--- TEST SPLIT ---

0    156
1     20
Name: Y, dtype: int64
0    0.886364
1    0.113636
Name: Y, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  176 non-null    int64 
 1   Drug     176 non-null    object
 2   Y        176 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB
None


,Drug_ID,Drug,Y
0,331,CCOc1ccc(NC(=O)NC(C)(C)C)cc1,0
1,247,C[C@H]1CN(C)CC[C@H]1CO,0
2,790,CC(C)C(=O)N1CCN(C(C)C)CC1,0
3,316,Cc1ccccc1C(=O)NC1CCN(C(=O)CCl)CC1,0
4,215,C[C@H]1NCCC[C@H]1C#N,0


### Separate actives and inactives

In [7]:
# Agreggate all actives.
touret_pos = pd.concat([train_touret[train_touret.Y == 1],
                        val_touret[val_touret.Y == 1],
                        test_touret[test_touret.Y == 1]])

diamond_pos = pd.concat([train_diamond[train_diamond.Y == 1],
                         val_diamond[val_diamond.Y == 1],
                         test_diamond[test_diamond.Y == 1]])
actives = pd.concat([touret_pos, diamond_pos])

In [8]:
print(actives.Y.value_counts())
print(actives.info())
display(actives.head())

1    166
Name: Y, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 169
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  166 non-null    int64 
 1   Drug     166 non-null    object
 2   Y        166 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.2+ KB
None


,Drug_ID,Drug,Y
0,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,1
1,1,C[C@]12C/C(=C/O)C(=O)C[C@@H]1CC[C@@H]1[C@@H]2C...,1
2,2,Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O.Cl,1
3,3,CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](O)CC[C...,1
4,4,C=C1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3C...,1


In [9]:
# Agreggate all inactives.
touret_neg = pd.concat([train_touret[train_touret.Y == 0],
                        val_touret[val_touret.Y == 0],
                        test_touret[test_touret.Y == 0]])

diamond_neg = pd.concat([train_diamond[train_diamond.Y == 0],
                         val_diamond[val_diamond.Y == 0],
                         test_diamond[test_diamond.Y == 0]])
inactives = pd.concat([touret_neg, diamond_neg])

In [10]:
print(inactives.Y.value_counts())
print(inactives.info())
display(inactives.head())

0    2198
Name: Y, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2198 entries, 62 to 175
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Drug_ID  2198 non-null   int64 
 1   Drug     2198 non-null   object
 2   Y        2198 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 68.7+ KB
None


,Drug_ID,Drug,Y
62,88,CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C(=O)O)C(C)(C)CCC1,0
63,89,CCC(COC(=O)c1cc(OC)c(OC)c(OC)c1)(c1ccccc1)N(C)C,0
64,90,Clc1ccc(Nc2nnc(Cc3ccncc3)c3ccccc23)cc1,0
65,91,CC(C)=CCC1C(=O)N(c2ccccc2)N(c2ccccc2)C1=O,0
66,92,CCN1CCCC(OC(=O)C(c2ccccc2)c2ccccc2)C1.Cl,0


## Train Hidden Markov Model, NGram, Combinatorial

### Fit models

Fit separate models for molecules that demonstrated activity and molecules that did not.

In [17]:
# Train HMM on actives.

start = datetime.now()
hmm_active = HMM()
hmm_active = hmm_active.fit(actives.Drug)
end = datetime.now()
print("HMM Time =", end-start)

start = datetime.now()
ngram_active = NGram()
ngram_active.fit(actives.Drug)
end = datetime.now()
print("NGram Time =", end-start)

start = datetime.now()
comb_active = CombinatorialGenerator()
comb_active.fit(actives.Drug)
end = datetime.now()
print("Comb Time =", end-start)


HMM Time = 0:22:30.710662
NGram Time = 0:00:00.072013
Comb Time = 0:00:00.093822


In [20]:
start = datetime.now()
hmm_inactive = HMM()
hmm_inactive = hmm_inactive.fit(inactives.Drug)
end = datetime.now()
print("HMM Time =", end-start)

start = datetime.now()
ngram_inactive = NGram()
ngram_inactive.fit(inactives.Drug)
end = datetime.now()
print("NGram Time =", end-start)

start = datetime.now()
comb_inactive = CombinatorialGenerator()
comb_inactive.fit(inactives.Drug)
end = datetime.now()
print("Comb Time =", end-start)

HMM Time = 5:39:23.626083
NGram Time = 0:00:00.928925
Comb Time = 0:00:01.183941


## Save trained models

In [21]:
# Pickle models.
path = "/home/yl759/cs6785/CS6785_project/data/pickled_model/"
hmm_active.save(path + "hmm_active.pkl")
hmm_inactive.save(path + "hmm_inactive.pkl")
ngram_active.save(path + "ngram_active.pkl")
ngram_inactive.save(path + "ngram_inactive.pkl")
comb_active.save(path + "comb_active.pkl")
comb_inactive.save(path + "comb_inactive.pkl")

In [ ]:
# # Test loading pickled models.
# test_hmm_active = HMM().load(path + active_name)
# test_hmm_inactive = HMM().load(path + inactive_name)
# print("ACTIVE SAMPLE =", test_hmm_active.generate_one())
# print("INACTIVE SAMPLE =", test_hmm_inactive.generate_one())

## Generate novel molecules

In [ ]:
# Generate 1000 de novo SARS-CoV-2 "actives."
de_novo_actives = []
for i in range(1000):
    de_novo = hmm_active.generate_one()
    de_novo_actives.append(de_novo)
    if i % 100 == 0:
        print(de_novo)
        
print(len(de_novo_actives))

In [ ]:
# Generate 1000 de novo SARS-CoV-2 "inactives."
de_novo_inactives = []
for i in range(1000):
    de_novo = hmm_inactive.generate_one()
    de_novo_inactives.append(de_novo)
    if i % 100 == 0:
        print(de_novo)
        
print(len(de_novo_inactives))

### Export de novo samples

In [ ]:
# Export as CSV.
df_de_novo_actives = pd.DataFrame({"SMILES": de_novo_actives,
                                   "Active": [1] * len(de_novo_actives)})
df_de_novo_inactives = pd.DataFrame({"SMILES": de_novo_inactives,
                                     "Active": [0] * len(de_novo_inactives)})
df_de_novo = pd.concat([df_de_novo_actives, df_de_novo_inactives])
df_de_novo.to_csv("hmm_de_novo_2k.csv", index = False)

In [ ]:
df_de_novo

In [ ]:
df_de_novo.Active.value_counts()

## Examine de novo molecule distributions